# Importing Libraries

In [1]:
import pandas as pd
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.metrics as skm

import plotly.figure_factory as ff

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"
pio.templates.default = 'plotly_dark'

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

import keras
from keras.models import Sequential
from keras.layers import Dense

# Import Training Data

In [2]:
init_time = time.time()
df = pd.read_csv("train_fare.csv")
fin_time = time.time()
print('Time Taken = ',fin_time-init_time)

Time Taken =  14.41999626159668


In [3]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dis,pick_jfk,pick_lga,pick_ewr,...,drop_ewr,lib,nyc,nss,sfs,Year,Month,Date,Day of Week,Hour
count,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,...,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06
mean,1.130765e+01,-7.397518e+01,4.075104e+01,-7.397433e+01,4.075141e+01,1.684524e+00,3.329680e+00,2.092487e+01,9.690042e+00,1.848865e+01,...,1.835533e+01,9.543668e+00,3.045107e+00,5.966981e+00,4.698996e+00,2.011736e+03,6.268651e+00,1.571701e+01,3.041605e+00,1.351137e+01
std,9.574454e+00,3.895083e-02,2.997922e-02,3.810609e-02,3.314532e-02,1.307684e+00,3.793898e+00,3.104879e+00,3.068253e+00,3.743363e+00,...,3.734315e+00,3.952870e+00,3.634363e+00,3.946078e+00,3.683681e+00,1.862968e+00,3.436545e+00,8.685090e+00,1.949380e+00,6.514874e+00
min,2.500000e+00,-7.498856e+01,4.003383e+01,-7.499828e+01,4.000565e+01,0.000000e+00,0.000000e+00,1.475508e-01,3.368652e-02,1.349275e-01,...,3.618007e-02,4.898866e-02,3.366094e-04,5.339799e-03,1.041811e-02,2.009000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+00,-7.399227e+01,4.073655e+01,-7.399158e+01,4.073560e+01,1.000000e+00,1.253349e+00,2.053507e+01,8.322832e+00,1.648999e+01,...,1.636911e+01,7.164995e+00,1.236304e+00,3.615365e+00,2.580605e+00,2.010000e+03,3.000000e+00,8.000000e+00,1.000000e+00,9.000000e+00
50%,8.500000e+00,-7.398210e+01,4.075335e+01,-7.398061e+01,4.075386e+01,1.000000e+00,2.152117e+00,2.118912e+01,9.524412e+00,1.797386e+01,...,1.782185e+01,9.129285e+00,2.331827e+00,5.601878e+00,4.181293e+00,2.012000e+03,6.000000e+00,1.600000e+01,3.000000e+00,1.400000e+01
75%,1.250000e+01,-7.396833e+01,4.076754e+01,-7.396536e+01,4.076840e+01,2.000000e+00,3.909590e+00,2.191394e+01,1.099023e+01,1.980174e+01,...,1.946658e+01,1.130821e+01,3.643930e+00,7.583171e+00,5.968077e+00,2.013000e+03,9.000000e+00,2.300000e+01,5.000000e+00,1.900000e+01
max,4.950000e+02,-7.206320e+01,4.192279e+01,-7.206700e+01,4.199811e+01,6.000000e+00,1.616862e+02,1.783438e+02,1.765042e+02,2.032123e+02,...,1.794818e+02,1.938528e+02,1.616659e+02,1.674743e+02,1.655208e+02,2.015000e+03,1.200000e+01,3.100000e+01,6.000000e+00,2.300000e+01


In [4]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dis,pick_jfk,...,drop_ewr,lib,nyc,nss,sfs,Year,Month,Date,Day of Week,Hour
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,1.030764,9.538054,...,28.056864,17.295010,11.983691,13.845911,11.952866,2009,6,15,0,17
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,8.450134,23.071126,...,13.575760,11.691725,6.065137,0.737101,2.654702,2010,1,5,1,16
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,1.389525,21.690922,...,18.022686,8.167039,0.551063,6.173413,4.742035,2011,8,18,3,0
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,2.799270,22.192143,...,16.521526,8.864549,2.832348,3.190886,1.608332,2012,4,21,5,4
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,1.999157,21.850063,...,19.460441,12.858502,1.427607,7.355035,5.688753,2010,3,9,1,7


In [5]:
df.shape

(4893757, 24)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4893757 entries, 0 to 4893756
Data columns (total 24 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
 8   dis                float64
 9   pick_jfk           float64
 10  pick_lga           float64
 11  pick_ewr           float64
 12  drop_jfk           float64
 13  drop_lga           float64
 14  drop_ewr           float64
 15  lib                float64
 16  nyc                float64
 17  nss                float64
 18  sfs                float64
 19  Year               int64  
 20  Month              int64  
 21  Date               int64  
 22  Day of Week        int64  
 23  Hour               int64  
dtypes: float64(16), int64(6), object(2)
memory usage: 

In [7]:
df.isnull().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dis                  0
pick_jfk             0
pick_lga             0
pick_ewr             0
drop_jfk             0
drop_lga             0
drop_ewr             0
lib                  0
nyc                  0
nss                  0
sfs                  0
Year                 0
Month                0
Date                 0
Day of Week          0
Hour                 0
dtype: int64

# EDA on Training DataSet

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['pickup_latitude'] = df['pickup_latitude'][(df['pickup_latitude'] >= 40) & (df['pickup_latitude'] <= 42)]
df['pickup_longitude'] = df['pickup_longitude'][(df['pickup_longitude'] <= -72) & (df['pickup_longitude'] >= -75)]
df['dropoff_latitude'] = df['dropoff_latitude'][(df['dropoff_latitude'] >= 40) & (df['dropoff_latitude'] <= 42)]
df['dropoff_longitude'] = df['dropoff_longitude'][(df['dropoff_longitude'] <= -72) & (df['dropoff_longitude'] >= -75)]

In [ ]:
df['pickup_longitude']= df['pickup_longitude'].replace({0:np.nan})
df['pickup_latitude']= df['pickup_latitude'].replace({0:np.nan})
df['dropoff_longitude']= df['dropoff_longitude'].replace({0:np.nan})
df['dropoff_latitude']= df['dropoff_latitude'].replace({0:np.nan})

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.describe()

In [ ]:
def sphere_dist(pick_lat, pick_lon, drop_lat, drop_lon):
    R_earth = 6371 # Earth radius (in km)
    # Convert degrees to radians
    pick_lat, pick_lon, drop_lat, drop_lon = map(np.radians, [pick_lat, pick_lon,
                                                              drop_lat, drop_lon])
    # Compute distances along lat, lon dimensions
    dlat = drop_lat - pick_lat
    dlon = drop_lon - pick_lon
    
    # Compute haversine distance
    a = np.sin(dlat/2.0)**2 + np.cos(pick_lat) * np.cos(drop_lat) * np.sin(dlon/2.0)**2
    return 2 * R_earth * np.arcsin(np.sqrt(a))

In [ ]:
init_time = time.time()
df['dis'] = sphere_dist( df['pickup_latitude'], df['pickup_longitude'],df['dropoff_latitude'], df['dropoff_longitude'])
df['pick_jfk'] = sphere_dist(40.6413, -73.7781, df['dropoff_latitude'], df['dropoff_longitude'])
df['pick_lga'] = sphere_dist(40.7769, -73.8740, df['dropoff_latitude'], df['dropoff_longitude'])
df['pick_ewr'] = sphere_dist(40.6895, -74.1745, df['dropoff_latitude'], df['dropoff_longitude'])
df['drop_jfk'] = sphere_dist(df['pickup_latitude'], df['pickup_longitude'], 40.6413, -73.7781)
df['drop_lga'] = sphere_dist(df['pickup_latitude'], df['pickup_longitude'], 40.7769, -73.8740)
df['drop_ewr'] = sphere_dist(df['pickup_latitude'], df['pickup_longitude'], 40.6895, -74.1745)
df['lib'] = sphere_dist(40.6892, -74.0445, df['dropoff_latitude'], df['dropoff_longitude'])
df['nyc'] = sphere_dist(df['pickup_latitude'], df['pickup_longitude'], 40.7577, -73.9782)
df['nss'] = sphere_dist(df['pickup_latitude'], df['pickup_longitude'], 40.7091, -74.0078)
df['sfs'] = sphere_dist(df['pickup_latitude'], df['pickup_longitude'], 40.7187, -73.9861)
fin_time = time.time()
print ('Time taken for exec : ',fin_time-init_time)

In [ ]:
df.describe()

In [ ]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

In [ ]:
df['Year'] = df['pickup_datetime'].dt.year
df['Month'] = df['pickup_datetime'].dt.month
df['Date'] = df['pickup_datetime'].dt.day
df['Day of Week'] = df['pickup_datetime'].dt.dayofweek
df['Hour'] = df['pickup_datetime'].dt.hour

In [ ]:
df.describe()

In [ ]:
df[(df['dis']==0)&(df['fare_amount']==0)]

In [ ]:
df = df.drop(index = df[(df['dis']==0) & (df['fare_amount']==0)].index, axis=0)

In [ ]:
df = df.drop(index= df[df['fare_amount']==0].index, axis=0)

In [ ]:
df = df.drop(index= df[df['fare_amount'] < 2.5].index, axis=0)

In [ ]:
df = df.drop(index= df[df['passenger_count'] >= 7].index, axis=0)

In [ ]:
fare_100 = df[df.fare_amount > 100]
fare_100.shape

In [ ]:
fare_100[fare_100['dis'] <1].shape

In [ ]:
df = df.drop(index= df[(df['dis'] <1) & (df['fare_amount'] > 100)].index, axis=0)

In [ ]:
df.describe()

In [ ]:
df.drop(df[df['fare_amount'] > 500].index, axis=0, inplace = True)

In [ ]:
df.describe()

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dis,pick_jfk,pick_lga,pick_ewr,...,drop_ewr,lib,nyc,nss,sfs,Year,Month,Date,Day of Week,Hour
count,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,...,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06,4.893757e+06
mean,1.130765e+01,-7.397518e+01,4.075104e+01,-7.397433e+01,4.075141e+01,1.684524e+00,3.329680e+00,2.092487e+01,9.690042e+00,1.848865e+01,...,1.835533e+01,9.543668e+00,3.045107e+00,5.966981e+00,4.698996e+00,2.011736e+03,6.268651e+00,1.571701e+01,3.041605e+00,1.351137e+01
std,9.574454e+00,3.895083e-02,2.997922e-02,3.810609e-02,3.314532e-02,1.307684e+00,3.793898e+00,3.104879e+00,3.068253e+00,3.743363e+00,...,3.734315e+00,3.952870e+00,3.634363e+00,3.946078e+00,3.683681e+00,1.862968e+00,3.436545e+00,8.685090e+00,1.949380e+00,6.514874e+00
min,2.500000e+00,-7.498856e+01,4.003383e+01,-7.499828e+01,4.000565e+01,0.000000e+00,0.000000e+00,1.475508e-01,3.368652e-02,1.349275e-01,...,3.618007e-02,4.898866e-02,3.366094e-04,5.339799e-03,1.041811e-02,2.009000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+00,-7.399227e+01,4.073655e+01,-7.399158e+01,4.073560e+01,1.000000e+00,1.253349e+00,2.053507e+01,8.322832e+00,1.648999e+01,...,1.636911e+01,7.164995e+00,1.236304e+00,3.615365e+00,2.580605e+00,2.010000e+03,3.000000e+00,8.000000e+00,1.000000e+00,9.000000e+00
50%,8.500000e+00,-7.398210e+01,4.075335e+01,-7.398061e+01,4.075386e+01,1.000000e+00,2.152117e+00,2.118912e+01,9.524412e+00,1.797386e+01,...,1.782185e+01,9.129285e+00,2.331827e+00,5.601878e+00,4.181293e+00,2.012000e+03,6.000000e+00,1.600000e+01,3.000000e+00,1.400000e+01
75%,1.250000e+01,-7.396833e+01,4.076754e+01,-7.396536e+01,4.076840e+01,2.000000e+00,3.909590e+00,2.191394e+01,1.099023e+01,1.980174e+01,...,1.946658e+01,1.130821e+01,3.643930e+00,7.583171e+00,5.968077e+00,2.013000e+03,9.000000e+00,2.300000e+01,5.000000e+00,1.900000e+01
max,4.950000e+02,-7.206320e+01,4.192279e+01,-7.206700e+01,4.199811e+01,6.000000e+00,1.616862e+02,1.783438e+02,1.765042e+02,2.032123e+02,...,1.794818e+02,1.938528e+02,1.616659e+02,1.674743e+02,1.655208e+02,2.015000e+03,1.200000e+01,3.100000e+01,6.000000e+00,2.300000e+01


# Train Test Split

In [10]:
x, y = df.drop(['fare_amount','key','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'], axis=1), df['fare_amount']

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=101)

# Model Training

### Linear Regression

In [12]:
lr = LinearRegression()

In [13]:
lr.fit(x_train, y_train)

LinearRegression()

In [14]:
pred = lr.predict(x_test)

In [15]:
print('Root Mean Squared Error :',np.sqrt(skm.mean_squared_error(y_test,pred)))

Root Mean Squared Error : 4.915105401136573


### Decision tree

In [16]:
dt = DecisionTreeRegressor(max_depth = 10)

In [17]:
dt.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=10)

In [18]:
dt_pred = dt.predict(x_test)

In [19]:
print('RMSE = ',np.sqrt(skm.mean_squared_error(y_test,dt_pred)))

RMSE =  3.8946610159744623


### Random Forest

In [20]:
rf = RandomForestRegressor(n_estimators = 20, min_samples_leaf = 60)

In [21]:
rf.fit(x_train, y_train)

RandomForestRegressor(min_samples_leaf=60, n_estimators=20)

In [22]:
rf_pred = rf.predict(x_test)

In [23]:
print('RMSE = ',np.sqrt(skm.mean_squared_error(y_test,rf_pred)))

RMSE =  3.602766740615941


### XG Boost

In [24]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_test, label=y_test)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

xgb_params = {
    'min_child_weight': 1, 
    'learning_rate': 0.05, 
    'colsample_bytree': 0.7, 
    'max_depth': 10,
    'subsample': 0.7,
    'n_estimators': 5000,
    'n_jobs': -1, 
    'booster' : 'gbtree', 
    'silent': 1,
    'eval_metric': 'rmse'}

model = xgb.train(xgb_params, dtrain, 700, watchlist, early_stopping_rounds=100, maximize=False, verbose_eval=50)

[23:16:16] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:13.76005	valid-rmse:13.80637
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[50]	train-rmse:3.58479	valid-rmse:3.78789
[100]	train-rmse:3.21816	valid-rmse:3.50426
[150]	train-rmse:3.10981	valid-rmse:3.45189
[200]	train-rmse:3.04071	valid-rmse:3.42468
[250]	train-rmse:2.99464	valid-rmse:3.40829
[300]	train-rmse:2.96081	valid-rmse:3.39741
[350]	train-rmse:2.92899	valid-rmse:3.38735
[400]	train-rmse:2.90192	valid-rmse:3.38153
[450]	train-rmse:2.88167	valid-rmse:3.37776
[500]	train-rmse:

### Keras

In [ ]:
n_cols = x_train.shape[1]

In [ ]:
n_cols

In [ ]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(28, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(28, activation='relu'))
    model.add(Dense(28, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
x_train = np.asarray(x_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')

In [ ]:
model = regression_model()

In [ ]:
model.fit(x_train, y_train, validation_split=0.3, epochs=30, verbose=2)

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# EDA On Test Data Set

In [37]:
test_df = pd.read_csv('test.csv')

In [38]:
test_df.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


In [39]:
def sphere_dist(pick_lat, pick_lon, drop_lat, drop_lon):
    R_earth = 6371 # Earth radius (in km)
    # Convert degrees to radians
    pick_lat, pick_lon, drop_lat, drop_lon = map(np.radians, [pick_lat, pick_lon,
                                                              drop_lat, drop_lon])
    # Compute distances along lat, lon dimensions
    dlat = drop_lat - pick_lat
    dlon = drop_lon - pick_lon
    
    # Compute haversine distance
    a = np.sin(dlat/2.0)**2 + np.cos(pick_lat) * np.cos(drop_lat) * np.sin(dlon/2.0)**2
    return 2 * R_earth * np.arcsin(np.sqrt(a))

In [40]:
init_time = time.time()
test_df['dis'] = sphere_dist( test_df['pickup_latitude'], test_df['pickup_longitude'],test_df['dropoff_latitude'], test_df['dropoff_longitude'])
test_df['pick_jfk'] = sphere_dist(40.6413, -73.7781, test_df['dropoff_latitude'], test_df['dropoff_longitude'])
test_df['pick_lga'] = sphere_dist(40.7769, -73.8740, test_df['dropoff_latitude'], test_df['dropoff_longitude'])
test_df['pick_ewr'] = sphere_dist(40.6895, -74.1745, test_df['dropoff_latitude'], test_df['dropoff_longitude'])
test_df['drop_jfk'] = sphere_dist(test_df['pickup_latitude'], test_df['pickup_longitude'], 40.6413, -73.7781)
test_df['drop_lga'] = sphere_dist(test_df['pickup_latitude'], test_df['pickup_longitude'], 40.7769, -73.8740)
test_df['drop_ewr'] = sphere_dist(test_df['pickup_latitude'], test_df['pickup_longitude'], 40.6895, -74.1745)
test_df['lib'] = sphere_dist(40.6892, -74.0445, test_df['dropoff_latitude'], test_df['dropoff_longitude'])
test_df['nyc'] = sphere_dist(test_df['pickup_latitude'], test_df['pickup_longitude'], 40.7577, -73.9782)
test_df['nss'] = sphere_dist(test_df['pickup_latitude'], test_df['pickup_longitude'], 40.7091, -74.0078)
test_df['sfs'] = sphere_dist(test_df['pickup_latitude'], test_df['pickup_longitude'], 40.7187, -73.9861)
fin_time = time.time()
print ('Time taken for exec : ',fin_time-init_time)

Time taken for exec :  0.023019790649414062


In [41]:
test_df['pickup_datetime'] = pd.to_datetime(test_df['pickup_datetime'])

In [42]:
test_df['Year'] = test_df['pickup_datetime'].dt.year
test_df['Month'] = test_df['pickup_datetime'].dt.month
test_df['Date'] = test_df['pickup_datetime'].dt.day
test_df['Day of Week'] = test_df['pickup_datetime'].dt.dayofweek
test_df['Hour'] = test_df['pickup_datetime'].dt.hour

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Predictions

### XG Boost Prediction

In [43]:
test = test_df.drop(['key','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'], axis=1)

In [44]:
dtest = xgb.DMatrix(test)

In [45]:
test_preds = model.predict(dtest)

In [46]:
test_preds

array([10.671461 , 10.93876  ,  4.3585606, ..., 53.835224 , 19.932985 ,
        7.0817323], dtype=float32)

In [47]:
preds = test_preds
filename = 'submissionxg.csv'#file name

key = test_df.loc[:,'key']
submission = pd.DataFrame(
    {'key': key, 'fare_amount': preds},
    columns = ['key', 'fare_amount'])
submission.to_csv(filename, index = False)

### Keras Prediction

In [ ]:
test_predictors = test_df.drop(['key','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'], axis=1)

In [ ]:
kfin_test_df = np.asarray(test_predictors).astype('float32')
predictions = model.predict(kfin_test_df)

In [ ]:
preds=[]
for i in predictions :
    preds.append(i[0])
preds = pd.DataFrame(preds, columns = ['Fare Amount'])
preds.fillna(0,inplace = True)
preds = preds.loc[:,'Fare Amount']

In [ ]:
preds = preds
filename = 'submissionk.csv'#file name

key = test_df.loc[:,'key']
submission = pd.DataFrame(
    {'key': key, 'fare_amount': preds},
    columns = ['key', 'fare_amount'])
submission.to_csv(filename, index = False)

### Prediction File Representation

In [48]:
pd.read_csv('submissionxg.csv').head()

,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.671461
1,2015-01-27 13:08:24.0000003,10.938760
2,2011-10-08 11:53:44.0000002,4.358561
3,2012-12-01 21:12:12.0000002,8.995962
4,2012-12-01 21:12:12.0000003,16.456976
